In [90]:
import pandas as pd
import re
from ydata_profiling import ProfileReport
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import shapiro


import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import shapiro
import pandas as pd

import matplotlib
matplotlib.use('TkAgg')


def check_normality_all_print(df):
    # Loop over all columns in the DataFrame
    for feature in df.columns:
        # Histogram
        plt.figure(figsize=(6, 6))
        plt.hist(df[feature].dropna(), edgecolor='black', bins=20)
        plt.title(f'Histogram of {feature}')
        plt.tight_layout()  # Auto-adjust layout for readability
        plt.show()  # Display the histogram
        
        # QQ plot
        plt.figure(figsize=(6, 6))
        sm.qqplot(df[feature].dropna(), line='45')
        plt.title(f'QQ plot of {feature}')
        plt.tight_layout()  # Auto-adjust layout for readability
        plt.show()  # Display the QQ plot





/var/folders/_g/_68qf0jx3dv_zdyngnnzdwc40000gn/T/ipykernel_95024/2754532269.py:17: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed in 3.10.  To suppress this warning, explicitly call plt.close('all') first.
  matplotlib.use('TkAgg')


In [91]:
def extract_int(x):
    digits = re.sub(r'\D', '', x)
    return int(digits) if digits else 0

def convert_quarter_string(s):
    s = s.strip()
    # Expected pattern: "YYYY <number>[optional letter] kwartaal"
    pattern = r"^(\d{4})\s+(\d+)[a-zA-Z]*\s+kwartaal$"
    m = re.fullmatch(pattern, s)
    if m:
        year = m.group(1)
        quarter = m.group(2)
        return pd.Period(f"{year}Q{quarter}", freq='Q')
    else:
        return None

def convert_year_string(s):
    s = s.strip()
    # Expected: exactly a four-digit year.
    m = re.fullmatch(r"\d{4}", s)
    if m:
        return int(s)
    else:
        return None
    
def convert_year_month_string(s):
    # If the value is already a Period, convert it directly.
    if isinstance(s, pd.Period):
        return s.to_timestamp(how='end')
    
    s = str(s).strip().lower()  # Normalize the string (e.g., lowercase for Dutch month names)
    
    # Mapping for Dutch month names to month numbers
    dutch_months = {
        'januari': '01',
        'februari': '02',
        'maart': '03',
        'april': '04',
        'mei': '05',
        'juni': '06',
        'juli': '07',
        'augustus': '08',
        'september': '09',
        'oktober': '10',
        'november': '11',
        'december': '12'
    }
    
    # First, try the direct conversion to a Period with monthly frequency.
    try:
        period_obj = pd.Period(s, freq='M')
        return period_obj
    except Exception:
        pass

    # New branch: try to match format like "1960M02"
    m = re.match(r"(\d{4})m(\d{1,2})", s)
    if m:
        year = int(m.group(1))
        month = int(m.group(2))
        period_obj = pd.Period(f"{year}-{month:02d}", freq='M')
        return period_obj

    # Try to match numeric year-month (formats like "2025-01" or "2025 1")
    m = re.match(r"(\d{4})[-\s]?(\d{1,2})", s)
    if m:
        year = int(m.group(1))
        month = int(m.group(2))
        period_obj = pd.Period(f"{year}-{month:02d}", freq='M')
        return period_obj
    
    # Try to match Dutch month names (formats like "2025 januari")
    m = re.match(r"(\d{4})\s*([a-zA-Z]+)", s)
    if m:
        year = int(m.group(1))
        month_str = m.group(2)
        month_num = dutch_months.get(month_str.lower())
        if month_num:
            period_obj = pd.Period(f"{year}-{month_num}", freq='M')
            return period_obj
    
    # If none of the above worked, return None.
    return None

    
def expand_quarter_to_daily(df, quarter_col=None, use_index=False):

    expanded_rows = []
    
    for idx, row in df.iterrows():
        # Extract the quarter string either from the index or the specified column.
        if use_index:
            quarter_str = str(idx).strip()
        else:
            if quarter_col is None:
                raise ValueError("quarter_col must be provided if use_index is False.")
            quarter_str = str(row[quarter_col]).strip()
        
        # Skip rows that should be dropped.
        if quarter_str == "Bron: CBS":
            continue
        
        period = convert_quarter_string(quarter_str)
        if period is None:
            continue  # Skip rows not matching the expected quarter format.
        
        start_date = period.start_time
        end_date = period.end_time
        
        # Create a daily date range for the quarter.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        # Duplicate the row for each day.
        row_expanded = pd.DataFrame([row] * len(daily_index), index=daily_index)
        row_expanded['date'] = daily_index
        
        expanded_rows.append(row_expanded)
    
    return pd.concat(expanded_rows) if expanded_rows else pd.DataFrame()



def expand_year_to_daily(df, year_col=None, use_index=False):
    expanded_rows = []
    
    for idx, row in df.iterrows():
        # Extract the year string either from the index or from the specified column.
        if use_index:
            year_str = str(idx).strip()
        else:
            if year_col is None:
                raise ValueError("year_col must be provided if use_index is False.")
            year_str = str(row[year_col]).strip()
        
        year_int = convert_year_string(year_str)
        if year_int is None:
            continue  # Skip rows that do not have a valid year.
        
        start_date = pd.Timestamp(year=year_int, month=1, day=1)
        end_date = pd.Timestamp(year=year_int, month=12, day=31)
        
        # Create a daily date range for the year.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        # Duplicate the row for each day.
# Convert the row to a dictionary and then create a DataFrame by replicating the dictionary values.
        row_dict = row.to_dict()
        row_expanded = pd.DataFrame({key: [value] * len(daily_index) for key, value in row_dict.items()},
                                    index=daily_index)
        row_expanded['date'] = daily_index
        
        expanded_rows.append(row_expanded)
    
    return pd.concat(expanded_rows) if expanded_rows else pd.DataFrame()

def expand_year_month_to_daily(df, year_month_col=None, use_index=False):
    expanded_rows = []

    for idx, row in df.iterrows():
        # Extract the year-month string either from index or a specified column.
        if use_index:
            year_month_str = str(idx).strip()
        else:
            if year_month_col is None:
                raise ValueError("year_month_col must be provided if use_index is False")
            year_month_str = str(row[year_month_col]).strip()
        
        period = convert_year_month_string(year_month_str)
        if period is None:
            continue  # Skip rows that don't have a valid year-month
        
        # Determine start and end dates for the month.
        start_date = period.start_time  # first day of the month
        end_date = period.end_time      # last day of the month

        # Create a daily date range for that month.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        
        # Duplicate the row for each day.
        # Instead of repeating the row (which can cause duplicate-index issues),
        # we create a dictionary with each column's value repeated.
        row_dict = row.to_dict()
        daily_df = pd.DataFrame({key: [value] * len(daily_index) for key, value in row_dict.items()},
                                index=daily_index)
        # Optionally add a 'date' column.
        daily_df['date'] = daily_index
        
        expanded_rows.append(daily_df)
    
    if expanded_rows:
        return pd.concat(expanded_rows)
    else:
        return pd.DataFrame()




In [ ]:

LaborParticipation = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Arbeidsdeelname__kerncijfers_seizoengecorrigeerd_21022025_095354.csv',
sep=';', header=3)


LaborParticipation = LaborParticipation.drop([0])
LaborParticipation

# #generating a report
def LaborParticipationPreprocess(LaborParticipation):
    LaborParticipation['Unnamed: 0'] = LaborParticipation['Unnamed: 0'].apply(lambda x: convert_quarter_string(x))
    LaborParticipation['Unnamed: 0'] = (
        pd.PeriodIndex(LaborParticipation['Unnamed: 0'], freq="Q")
        .to_timestamp()  # Converts Period to datetime
    )
    LaborParticipation.set_index('Unnamed: 0', inplace=True)
    LaborParticipation.index.names = ['Date']
    # In the data full years could be skipped, due to the conversion these became null thus we can drop them
    LaborParticipation = LaborParticipation[LaborParticipation.index.notnull()]
    return LaborParticipation


print(check_normality_all_print(LaborParticipationPreprocess(LaborParticipation)))





2025-03-01 20:53:08.318 python[95024:16968517] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-01 20:53:08.318 python[95024:16968517] +[IMKInputSession subclass]: chose IMKInputSession_Modern


None


: 

In [66]:
ConsumerPriceIndex = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Consumentenprijzen__prijsindex_2015_100_01032025_140302.csv',sep=';', header=3)
ConsumerPriceIndex.drop([0])

def ConsumerPriceIndexPreprocess(ConsumerPriceIndex):
    ConsumerPriceIndex.columns += ' | ' + ConsumerPriceIndex.iloc[0]
    ConsumerPriceIndex = ConsumerPriceIndex.drop(0)
    
    ConsumerPriceIndex['Unnamed: 0 | Perioden'] = ConsumerPriceIndex['Unnamed: 0 | Perioden'].apply(lambda x: convert_year_month_string(x))
    ConsumerPriceIndex.set_index(['Unnamed: 0 | Perioden'], inplace=True)
    ConsumerPriceIndex.index.names = ['Date']
    
    return ConsumerPriceIndex

# ConsumerPriceIndexPreprocess(ConsumerPriceIndex)
check_normality_all(ConsumerPriceIndexPreprocess(ConsumerPriceIndex))

# ConsumerPriceIndex_daily = expand_year_to_daily(ConsumerPriceIndex, 'Unnamed: 0')
# ConsumerPriceIndex_daily.drop(['date'],axis=1,inplace=True)




Total population size by age per year. Added a with the average age

In [67]:
PopulationSize = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bevolking_op_1_januari_en_gemiddeld__geslacht__leeftijd_en_regio_23022025_161215.csv',sep=';',header=4)
PopulationSize = PopulationSize.drop(['Unnamed: 0'],axis=1)



def PopulationSizePreprocess(PopulationSize):
    PopulationSize.iloc[1, 2:] = PopulationSize.iloc[1, 2:].apply(extract_int)
    # Set the second row as header
    PopulationSize.columns = PopulationSize.iloc[1]
    # Drop the first two rows (the previous header and the row used as header) and reset the index
    PopulationSize = PopulationSize.iloc[2:].reset_index(drop=True)
    PopulationSize.drop([0], inplace=True)
    PopulationSize.rename(columns={'Leeftijd':'Date'}, inplace=True)
    PopulationSize.set_index('Date', inplace=True)
    return PopulationSize













In [68]:
Gdp = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Opbouw_binnenlands_product__bbp___nationale_rekeningen__1995_2022_28022025_221707.csv',sep=';', header=3).T

# Gdp_daily = expand_year_to_daily(Gdp,use_index=True )
Gdp.rename(columns={'Onderwerp':'Date'}, inplace=True)
# Gdp_daily

Gdp


,0,1,2,3,4
Onderwerp,Bbp vanuit de finale bestedingen|Waarde in wer...,Bbp vanuit de finale bestedingen|Waarde in wer...,Bbp vanuit de finale bestedingen|Waarde in wer...,Bbp vanuit de finale bestedingen|Waarde in wer...,Bron: CBS
Unnamed: 1,mln euro,mln euro,mln euro,mln euro,NaN
1995,166743.0,129241.0,37502.0,329547.0,NaN
1996,176946.0,136621.0,40325.0,344625.0,NaN
1997,199214.0,152448.0,46766.0,369046.0,NaN
1998,209811.0,158262.0,51549.0,394295.0,NaN
1999,228592.0,169612.0,58980.0,419459.0,NaN
2000,268893.0,203973.0,64920.0,452007.0,NaN
2001,273421.0,203301.0,70120.0,481881.0,NaN
2002,267151.0,193376.0,73775.0,501137.0,NaN


In [69]:
GdpProductionAndSpending = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bbp__productie_en_bestedingen__kwartalen__mutaties__na__1995__2024_I_28022025_214426.csv',sep=';', header=3).T
GdpProductionAndSpending = GdpProductionAndSpending.reset_index(drop=True)
# set header of
GdpProductionAndSpending.columns = GdpProductionAndSpending.iloc[0]

GdpProductionAndSpending = GdpProductionAndSpending.drop('Bron: CBS', axis=1)   
GdpProductionAndSpending_daily = expand_quarter_to_daily(GdpProductionAndSpending, 'Onderwerp')




In [70]:
Boughthomes =  pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bestaande_koopwoningen__verkoopprijzen_prijsindex_2015_100_1995_2023_25022025_192415.csv',sep=';', header=3)

# Boughthomes_daily = expand_year_month_to_daily(Boughthomes, year_month_col="Unnamed: 0")
Boughthomes['Unnamed: 0'] = Boughthomes['Unnamed: 0'].apply(lambda x: convert_year_month_string(x))
Boughthomes.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
Boughthomes.set_index(['Date'], inplace=True)
Boughthomes = Boughthomes.iloc[1:]

Boughthomes



,Prijsindex verkoopprijzen|Prijsindex bestaande koopwoningen,Prijsindex verkoopprijzen|Ontwikkeling t.o.v. voorgaande periode,Prijsindex verkoopprijzen|Ontwikkeling t.o.v. een jaar eerder,Aantal verkochte woningen|Aantal verkochte woningen,Aantal verkochte woningen|Ontwikkeling t.o.v. voorgaande periode,Aantal verkochte woningen|Ontwikkeling t.o.v. een jaar eerder,Gemiddelde verkoopprijs,Totale waarde verkoopprijzen
Date,,,,,,,,
1995-01,"41,3",NaN,NaN,8793,NaN,NaN,88991,782
1995-02,"41,5","0,6",NaN,10103,"14,9",NaN,89741,907
1995-03,"41,4","-0,3",NaN,11838,"17,2",NaN,90432,1071
1995-04,"41,6","0,5",NaN,10626,"-10,2",NaN,89808,954
1995-05,"41,8","0,6",NaN,12746,"20,0",NaN,92326,1177
...,...,...,...,...,...,...,...,...
2023-10,"182,4","0,7","-2,3",15705,"-2,4","5,7",424521,6667
2023-11,"183,2","0,5","-0,9",15248,"-2,9","0,7",421284,6424
2023-12,"183,5","0,2","1,6",19565,"28,3","-8,3",422372,8264


In [71]:
def find_columns_energy_commodities(df):
    # Define a pattern that matches any of the keywords
    pattern = r"(Coal|Coal (Colombia)|Coal (South Africa)|Crude oil| index (Laspeyres)|Natural Gas|natural gas)"
    # Use a list comprehension to check each column name
    matching_columns = [col for col in df.columns if re.search(pattern, str(col), re.IGNORECASE)]

    return matching_columns

# Commodities_daily = expand_year_month_to_daily(Commodities, year_month_col="Unnamed: 0")
# Commodities_daily.drop(['Unnamed: 0','date'],axis=1,inplace=True)
Commodities = pd.read_excel('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/Commodity Future Data/CMO/CMO-Historical-Data-Monthly.xlsx',sheet_name='Monthly Prices',skiprows=4)
Commodities['Unnamed: 0'] = Commodities['Unnamed: 0'].apply(lambda x: convert_year_month_string(x))
Commodities.set_index('Unnamed: 0', inplace=True)
Commodities.index.names = ['Date']
Commodities.columns += ' | ' + Commodities.iloc[0]
Commodities = Commodities.iloc[1:]

Commodities[find_columns_energy_commodities(Commodities)]



,"Crude oil, average | ($/bbl)","Crude oil, Brent | ($/bbl)","Crude oil, Dubai | ($/bbl)","Crude oil, WTI | ($/bbl)","Coal, Australian | ($/mt)","Coal, South African ** | ($/mt)","Natural gas, US | ($/mmbtu)","Natural gas, Europe | ($/mmbtu)","Liquefied natural gas, Japan | ($/mmbtu)",Natural gas index | (2010=100)
Date,,,,,,,,,,
1960-01,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,…
1960-02,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,…
1960-03,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,…
1960-04,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,…
1960-05,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,…
...,...,...,...,...,...,...,...,...,...,...
2024-08,78.121,80.863,77.95,75.55,145.76,106.21,1.9903,12.374598,13.319637,92.021428
2024-09,72.424333,74.293,73.43,69.55,139.2,106.07,2.2508,11.784259,12.969226,92.140921
2024-10,73.970667,75.662,74.65,71.6,146.63,106.81,2.2077,12.920133,12.54098,96.72732


In [47]:
# Reset index to turn the date index into a column
# Will embed this into the function I will make for each individual dataframe.
# Gdp_daily_reset = Gdp.reset_index()
# GdpProductionAndSpending_reset = GdpProductionAndSpending.reset_index()
# PopulationSize_reset = PopulationSize.reset_index()
# ConsumerPriceIndex_reset = ConsumerPriceIndex.reset_index()
# LaborParticipation_reset = LaborParticipation.reset_index()
# Boughthomes_dreset = Boughthomes.reset_index()
# Commodities_reset = Commodities.reset_index()



# Merge the dataframes on the 'index' column (which holds the dates)
combined_df = GdpProductionAndSpending.merge(PopulationSize, on='Date', how='outer') \
                         .merge(ConsumerPriceIndex,on='Date', how='outer')\
                         .merge(LaborParticipation,on='Date', how='outer')\
                         .merge(Boughthomes,on='Date', how='outer')\
                         .merge(Commodities,on='Date', how='outer')

# Optionally, set the 'index' column back as the index
combined_df.set_index('index', inplace=True)


combined_df.to_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/combined_df.csv')

# Define the constant comparison date

combined_df


KeyError: 'Date'

In [ ]:
combined_df[combined_df.index >= '1996-01-01']

,Bbp vanuit de finale bestedingen|Waarde in werkelijke prijzen|Invoer van goederen en diensten (-)|Totaal,Bbp vanuit de finale bestedingen|Waarde in werkelijke prijzen|Invoer van goederen en diensten (-)|Goederen,Bbp vanuit de finale bestedingen|Waarde in werkelijke prijzen|Invoer van goederen en diensten (-)|Diensten,Bbp vanuit de finale bestedingen|Waarde in werkelijke prijzen|Bruto binnenlands product,Bron: CBS,date_x,Onderwerp,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Bruto binnenlands product,"Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Bbp, gecorrigeerd voor werkdageneffecten",...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
index,,,,,,,,,,,,,,,,,,,,,
1996-01-01,176946.0,136621.0,40325.0,344625.0,NaN,1996-01-01,1996 1e kwartaal,"3,5","2,4",.,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-01,176946.0,136621.0,40325.0,344625.0,NaN,1996-01-01,1996 1e kwartaal,.,.,.,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-02,176946.0,136621.0,40325.0,344625.0,NaN,1996-01-02,1996 1e kwartaal,"3,5","2,4",.,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-02,176946.0,136621.0,40325.0,344625.0,NaN,1996-01-02,1996 1e kwartaal,.,.,.,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-03,176946.0,136621.0,40325.0,344625.0,NaN,1996-01-03,1996 1e kwartaal,"3,5","2,4",.,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2541.02,102.21,8916.32,1990.43,28864.99,15444.89,3034.16,2648.01,937.88,30.7640
2024-12-28,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2541.02,102.21,8916.32,1990.43,28864.99,15444.89,3034.16,2648.01,937.88,30.7640
2024-12-29,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2541.02,102.21,8916.32,1990.43,28864.99,15444.89,3034.16,2648.01,937.88,30.7640


In [ ]:
cutoff_date = pd.Timestamp('1996-01-01')
filtered_df = combined_df[combined_df.index >= cutoff_date]

In [ ]:
filtered_df

,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Totaal,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Lonen,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Sociale premies t.l.v. werkgevers,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Saldo,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Belastingen op productie en invoer,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Subsidies,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Bruto,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Verbruik van vaste activa (-),Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Netto,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Bruto binnenlands product,...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
date,,,,,,,,,,,,,,,,,,,,,
1996-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
1996-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1589.34,30.00,2616.41,709.50,6271.60,7862.05,1019.10,399.45,415.48,5.4605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2541.02,102.21,8916.32,1990.43,28864.99,15444.89,3034.16,2648.01,937.88,30.7640
2024-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2541.02,102.21,8916.32,1990.43,28864.99,15444.89,3034.16,2648.01,937.88,30.7640
2024-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2541.02,102.21,8916.32,1990.43,28864.99,15444.89,3034.16,2648.01,937.88,30.7640
